# Avaliador Acadêmico com LLMs e Langgraph no Google Colab

Este notebook permite executar o sistema de avaliação acadêmica diretamente no Google Colab. Siga os passos abaixo para configurar e rodar o avaliador.

## Passo 1: Fazer Upload do Pacote do Projeto (`academic_evaluator.zip`)

Execute a célula abaixo. Será solicitado que você escolha o arquivo `academic_evaluator.zip` que foi fornecido a você (contendo todo o projeto).

In [ ]:
from google.colab import files

print("Por favor, faça o upload do arquivo academic_evaluator.zip")
uploaded = files.upload()

for fn in uploaded.keys():
  print(f"Arquivo 	"{fn}" de {len(uploaded[fn])} bytes carregado com sucesso.")
  PROJECT_ZIP_NAME = fn # Salva o nome do arquivo zip para o próximo passo

## Passo 2: Descompactar o Projeto

Esta célula irá descompactar o arquivo `academic_evaluator.zip` que você acabou de carregar.

In [ ]:
import zipfile
import os

if "PROJECT_ZIP_NAME" in locals() and os.path.exists(PROJECT_ZIP_NAME):
    print(f"Descompactando {PROJECT_ZIP_NAME}...")
    extract_to_dir = "." # Descompacta na raiz do Colab /content/
    with zipfile.ZipFile(PROJECT_ZIP_NAME, "r") as zip_ref:
        zip_ref.extractall(extract_to_dir)
    print(f"Projeto descompactado com sucesso em {os.path.abspath(extract_to_dir)}.")
    # Verifica se o diretório principal do projeto existe após a descompactação
    if os.path.isdir("academic_evaluator"):
        print("Diretório 'academic_evaluator' encontrado.")
    else:
        print("ATENÇÃO: Diretório 'academic_evaluator' não encontrado após descompactação. Verifique o conteúdo do ZIP.")
else:
  print("Erro: O arquivo do projeto ZIP não foi carregado ou não foi encontrado. Execute o Passo 1 primeiro.")

## Passo 3: Instalar as Dependências

Agora, vamos instalar todas as bibliotecas Python necessárias listadas no arquivo `requirements.txt` do projeto.

In [ ]:
import os

REQUIREMENTS_PATH = "academic_evaluator/requirements.txt"

if os.path.exists(REQUIREMENTS_PATH):
  print(f"Instalando dependências de {REQUIREMENTS_PATH}...")
  !pip install -q -r $REQUIREMENTS_PATH
  print("Dependências instaladas.")
else:
  print(f"Erro: Arquivo {REQUIREMENTS_PATH} não encontrado. Certifique-se de que o projeto foi descompactado corretamente e que o arquivo existe.")

## Passo 4: Configurar as Chaves de API

Execute a célula abaixo. Você será solicitado a inserir suas chaves de API para OpenAI e Gemini. Estas chaves são necessárias para que os modelos de linguagem possam ser acessados.

**Importante**: Suas chaves de API serão usadas apenas nesta sessão do Colab e não serão armazenadas permanentemente no notebook.

In [ ]:
import os
from getpass import getpass

print("Por favor, insira sua chave de API da OpenAI:")
openai_api_key = getpass()
os.environ["OPENAI_API_KEY"] = openai_api_key

print("
Por favor, insira sua chave de API do Gemini (Google AI Studio):")
gemini_api_key = getpass()
os.environ["GEMINI_API_KEY"] = gemini_api_key

print("
Chaves de API configuradas como variáveis de ambiente para esta sessão.")

## Passo 5: Verificar a Estrutura de Diretórios (Opcional)

Esta célula lista o conteúdo do diretório `academic_evaluator` para confirmar que tudo foi configurado corretamente.

In [ ]:
import os

PROJECT_DIR = "academic_evaluator"
if os.path.isdir(PROJECT_DIR):
  print(f"Conteúdo do diretório {PROJECT_DIR}:")
  !ls -R $PROJECT_DIR
else:
  print(f"Erro: Diretório {PROJECT_DIR} não encontrado.")

## Passo 6: Executar a Avaliação

Esta célula executará o script principal do avaliador acadêmico. Ele processará os PDFs no diretório `academic_evaluator/pdfs/` e gerará um relatório CSV.

**Nota**: A execução pode levar alguns minutos, dependendo do número de trabalhos e da complexidade das avaliações.
Certifique-se de que os arquivos PDF de exemplo (ou os seus próprios) estão no diretório `academic_evaluator/pdfs/` e o arquivo `State_of_AI_Report_2024.pptx` (ou o seu de referência) está em `academic_evaluator/reference_materials/` antes de executar.

In [ ]:
import os
import sys

# Adiciona o diretório do projeto ao sys.path para que as importações funcionem
# O diretório raiz do Colab é /content/, e o projeto é descompactado em /content/academic_evaluator
base_project_dir = os.path.abspath(".") # /content/
project_module_dir = os.path.join(base_project_dir, "academic_evaluator") # /content/academic_evaluator

if project_module_dir not in sys.path:
    sys.path.insert(0, project_module_dir)
# Adicionar o diretório pai de src para que `from academic_evaluator.src...` funcione
if base_project_dir not in sys.path:
    sys.path.insert(0, base_project_dir)

PDF_DIR = "academic_evaluator/pdfs"
CONFIG_FILE = "academic_evaluator/config/criteria.json"
REPORTS_DIR = "academic_evaluator/reports"
REF_MATERIALS_DIR = "academic_evaluator/reference_materials"

# Criar diretórios se não existirem (especialmente reports e os de input)
os.makedirs(PDF_DIR, exist_ok=True)
os.makedirs(REPORTS_DIR, exist_ok=True)
os.makedirs(REF_MATERIALS_DIR, exist_ok=True)

print(f"Verificando diretório de PDFs: {os.path.abspath(PDF_DIR)}")
if not os.listdir(PDF_DIR):
    print(f"ATENÇÃO: Diretório de PDFs {PDF_DIR} está vazio. A avaliação pode não encontrar arquivos para processar.")
    print(f"Por favor, adicione arquivos PDF ao diretório {PDF_DIR} no painel esquerdo do Colab (Files -> Upload to {PDF_DIR}) e re-execute esta célula se necessário.")

print(f"Verificando arquivo de configuração: {os.path.abspath(CONFIG_FILE)}")
if not os.path.exists(CONFIG_FILE):
    print(f"ERRO CRÍTICO: Arquivo de configuração {CONFIG_FILE} não encontrado. A avaliação não pode prosseguir.")
else:
    print("
Iniciando o processo de avaliação...")
    # Chamada ao script principal como módulo. 
    # Os caminhos dos diretórios são relativos à raiz do Colab onde 'academic_evaluator' foi descompactado.
    command = f"python -m academic_evaluator.src.main --pdf_dir {PDF_DIR} --config_file {CONFIG_FILE} --reports_dir {REPORTS_DIR} --ref_materials_dir {REF_MATERIALS_DIR}"
    print(f"Executando comando: {command}")
    !{command}
    print("
Avaliação concluída.")

## Passo 7: Visualizar e Baixar o Relatório

Após a execução, os relatórios CSV são salvos no diretório `academic_evaluator/reports/`. Esta célula lista os relatórios disponíveis e permite baixar o mais recente.

In [ ]:
import os
import glob
from google.colab import files

REPORTS_DIR = "academic_evaluator/reports"

if os.path.isdir(REPORTS_DIR):
  report_files = glob.glob(os.path.join(REPORTS_DIR, "*.csv"))
  if report_files:
    latest_report = max(report_files, key=os.path.getctime)
    print(f"Relatórios encontrados em {os.path.abspath(REPORTS_DIR)}:")
    for rf in report_files:
      print(f"- {os.path.basename(rf)}")
    print(f"
O relatório mais recente é: {os.path.basename(latest_report)}")
    print("
Para baixar o relatório mais recente, execute a próxima célula ou clique com o botão direito no arquivo no painel esquerdo (Files) e escolha "Download".")
  else:
    print(f"Nenhum relatório CSV encontrado em {REPORTS_DIR}.")
else:
  print(f"Erro: Diretório de relatórios {REPORTS_DIR} não encontrado.")

In [ ]:
# Célula para download explícito do relatório mais recente (se existir)
if "latest_report" in locals() and os.path.exists(latest_report):
  print(f"Iniciando download de {os.path.basename(latest_report)}...")
  files.download(latest_report)
else:
  print("Nenhum relatório para baixar ou a célula anterior não foi executada/não encontrou relatórios.")

## Solução de Problemas e Dicas

- **`academic_evaluator` não encontrado**: Certifique-se de que você fez o upload do `academic_evaluator.zip` e executou a célula de descompactação (Passo 2) corretamente.
- **Chaves de API**: Se houver erros de autenticação com OpenAI ou Gemini, verifique se você inseriu as chaves corretamente no Passo 4.
- **Arquivos PDF/PPTX**: Para avaliar seus próprios trabalhos, faça o upload deles para o diretório `academic_evaluator/pdfs/` e, se necessário, materiais de referência para `academic_evaluator/reference_materials/` usando o painel de arquivos à esquerda no Colab. Depois, re-execute o Passo 6.
- **Modificar Critérios**: Para modificar os critérios de avaliação, você pode editar o arquivo `academic_evaluator/config/criteria.json` diretamente no Colab (clique duas vezes no arquivo no painel de arquivos) antes de executar o Passo 6.
- **Erro de Importação de Módulo**: Se você encontrar erros como `ModuleNotFoundError` ao executar o Passo 6, certifique-se de que o Passo 2 (descompactação) foi concluído com sucesso e o diretório `academic_evaluator` está presente na raiz do seu ambiente Colab (`/content/academic_evaluator`). A célula de execução da avaliação tenta adicionar os caminhos corretos ao `sys.path`, mas a estrutura de diretórios deve estar correta.